# Лабораторная работа 4, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [2]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [3]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [4]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [5]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_28091/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1929,0.276774,-0.018056,-0.11455,-0.988543,-0.973109,-0.968738,-0.990357,-0.975245,-0.968332,-0.924358,...,0.254551,-0.386452,-0.113445,-0.676951,-0.561829,-0.153955,-0.154682,11,4,SITTING


Подготовим тестовую выборку

In [6]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_28091/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1445,0.270979,-0.016331,-0.09974,-0.993985,-0.993469,-0.986863,-0.994522,-0.993889,-0.986857,-0.935918,...,0.069981,0.024661,0.297225,-0.539833,0.530514,-0.838667,0.184747,12,6,LAYING


Поменяем названия функций

In [7]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [8]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [9]:
def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

### a. Обучить модели со случайным лесом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

classifier = RandomForestClassifier()

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9250
Recall: 0.9250
Precision: 0.9261
F1 Score: 0.9248


### b. Обучить модели со случайным лесом из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = RandomForestRegressor()

regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 3.00
MSE: 69.60


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Количество деревьев (n_estimators): Определяет число деревьев в лесу. Большее число деревьев увеличивает стабильность модели, но замедляет обучение.
    - Максимальная глубина деревьев (max_depth): Контролирует максимальную глубину каждого дерева. Меньшая глубина снижает переобучение, но может ухудшить качество.
    - Минимальное количество образцов для разделения (min_samples_split): Минимальное количество данных, необходимое для разделения узла.
    - Минимальное количество образцов в листе (min_samples_leaf): Минимальное количество данных в каждом листе дерева.
    - Критерий разбиения (criterion): Для классификации может быть gini или entropy, а для регрессии — squared_error (по умолчанию) или absolute_error.
    - Максимальное количество признаков для выбора (max_features): Указывает, сколько признаков будет использоваться при построении каждого дерева. Значение может быть числовым, дробным (например, 0.5 — половина признаков), sqrt или log2.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Подбор гиперпараметров и Grid Search

#### Задача классификации

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

classifier = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(train_X, train_y)

best_classifier = grid_search.best_estimator_

y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации после оптимизации:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.6s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.6s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   3.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   3.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_e

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Результаты для задачи классификации после оптимизации:
Лучшие параметры: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.9342
Recall: 0.9342
Precision: 0.9364
F1 Score: 0.9338


#### Задача регрессии

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

regressor = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Оптимизация MSE
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

best_regressor = grid_search.best_estimator_

y_pred_test = best_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии после оптимизации:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_e

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Результаты для задачи регрессии после оптимизации:
Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
MAE: 2.65
MSE: 35.55


### Формирование новых признаков

#### Задача регрессии

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = RandomForestRegressor(random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.71
MSE: 46.22


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

classifier = RandomForestClassifier(max_depth= 20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=100)

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9416
Recall: 0.9416
Precision: 0.9431
F1 Score: 0.9411


#### Задача регрессии

Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = RandomForestRegressor(max_depth= 20, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 200)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.64
MSE: 33.98


### Выводы

После улучшения базовых моделей для задач классификации и регрессии с использованием Random Forest, удалось достичь заметного повышения качества. В задаче классификации внедрение оптимизации гиперпараметров привело к увеличению метрик точности, полноты, точности предсказания и F1-метрики. Это свидетельствует о повышении способности модели корректно классифицировать данные и снижении ошибок, связанных с неверной категоризацией. В задаче регрессии также удалось улучшить метрики за счет добавления нового признака и последующей оптимизации гиперпараметров. Снижение MAE и MSE указывает на более точное предсказание моделью целевого значения, что отражает ее улучшенную способность улавливать закономерности в данных. Таким образом, проведенные улучшения базовых моделей существенно повысили их производительность и продемонстрировали значимость правильного выбора признаков и настройки гиперпараметров для получения высоких результатов.

## 4. Имплементация алгоритма машинного обучения 

### Базовый класс

In [26]:
class DecisionTreeCustom:
    def __init__(self, max_depth=None, min_samples_split=2, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.tree = None

    def _gini_impurity(self, left_y, right_y):
        def gini(y):
            m = len(y)
            if m == 0:
                return 0
            p = np.bincount(y) / m
            return 1 - np.sum(p ** 2)

        total = len(left_y) + len(right_y)
        return (len(left_y) / total) * gini(left_y) + (len(right_y) / total) * gini(right_y)

    def _leaf_value(self, y):
        return Counter(y).most_common(1)[0][0]

    def _best_split(self, X, y):
        n_samples, n_features = X.shape

        if isinstance(self.max_features, str):
            if self.max_features == "sqrt":
                n_selected_features = int(np.sqrt(n_features))
            elif self.max_features == "log2":
                n_selected_features = int(np.log2(n_features))
            else:
                raise ValueError(f"Unknown max_features value: {self.max_features}")
        elif self.max_features is None:
            n_selected_features = n_features
        else:
            n_selected_features = self.max_features

        features = np.random.choice(n_features, n_selected_features, replace=False)

        best_gini = float("inf")
        best_split = None

        for feature in features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_split and len(right_y) >= self.min_samples_split:
                    gini = self._gini_impurity(left_y, right_y)
                    if gini < best_gini:
                        best_gini = gini
                        best_split = (feature, threshold, left_mask, right_mask)

        return best_split

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split or len(set(y)) == 1:
            return self._leaf_value(y)

        split = self._best_split(X, y)
        if split is None:
            return self._leaf_value(y)

        feature, threshold, left_mask, right_mask = split
        left_tree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {"feature": feature, "threshold": threshold, "left": left_tree, "right": right_tree}

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, tree):
        if not isinstance(tree, dict):
            return tree

        feature_value = x[tree["feature"]]
        if feature_value <= tree["threshold"]:
            return self._predict_sample(x, tree["left"])
        else:
            return self._predict_sample(x, tree["right"])

    def predict(self, X):
        return np.array([self._predict_sample(row, self.tree) for row in X])


### Имплементация случайного леса для классификации

In [27]:
class RandomForestClassifierCustom:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)

        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        majority_votes = np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=tree_preds)
        return majority_votes

### Имплементация случайного леса для регрессии

In [28]:
class RandomForestRegressorCustom:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)

        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [34]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = RandomForestClassifierCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features="sqrt", random_state=42
)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")


Accuracy: 0.9169, Recall: 0.9169, Precision: 0.9193, F1 Score: 0.9166


#### Задача регрессии

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features=5, random_state=42
)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")


MAE: 7.30, MSE: 238.79


### Сравнение результатов с п.2. Выводы

Сравнение результатов обучения моделей из библиотеки sklearn и самописных реализаций для задач классификации и регрессии показало, что модели из sklearn демонстрируют более высокую точность и лучшие метрики качества. В задаче классификации sklearn-модель достигла значения Accuracy 0.9250 и F1 Score 0.9248, тогда как самописная модель показала Accuracy 0.9169 и F1 Score 0.9166. Несмотря на то, что разница в результатах невелика, модели sklearn оказались более устойчивыми в своих предсказаниях, вероятно, благодаря более сложной внутренней реализации и оптимизациям. В задаче регрессии разрыв между результатами оказался более заметным: MAE и MSE для sklearn-модели составили 3.00 и 69.60 соответственно, в то время как для самописной модели эти метрики равнялись 7.30 и 238.79. Это указывает на то, что самописная реализация пока уступает библиотечным вариантам в точности предсказаний, что может быть связано с ограничениями в использовании усреднения, глубины деревьев или других аспектов алгоритма. Таким образом, библиотечные реализации более эффективны и удобны для использования в реальных задачах, но создание собственных моделей позволяет лучше понять принципы работы алгоритмов и их внутренние механизмы.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features=5, random_state=42
)
rf_regressor.fit(X_train_new_field, y_train_new_field)
y_pred_test = rf_regressor.predict(X_test_new_field)

mae = mean_absolute_error(y_test_new_field, y_pred_test)
mse = mean_squared_error(y_test_new_field, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 6.75, MSE: 207.63


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [44]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def grid_search_rf_classification(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = -np.inf

    for n_estimators in param_grid['n_estimators']:
        for min_samples_split in param_grid['min_samples_split']:
            for max_features in param_grid['max_features']:
                rf_classifier = RandomForestClassifierCustom(
                    n_estimators=n_estimators,
                    max_depth=10,
                    min_samples_split=min_samples_split,
                    max_features=max_features,
                    random_state=42
                )
                rf_classifier.fit(X_train, y_train)
                y_pred = rf_classifier.predict(X_test)
                
                accuracy = accuracy_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred, average='weighted')

                if f1 > best_score:
                    best_score = f1
                    best_params = {
                        'n_estimators': n_estimators,
                        'min_samples_split': min_samples_split,
                        'max_features': max_features
                    }

    print(f"Best parameters: {best_params}")
    print(f"Best F1 Score: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

best_params_classification = grid_search_rf_classification(train_X, train_y, test_X, test_y, param_grid)


Best parameters: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'log2'}
Best F1 Score: 0.9248


##### Задача регрессии

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def grid_search_rf_regression(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for min_samples_split in param_grid['min_samples_split']:
                for max_features in param_grid['max_features']:
                    rf_regressor = RandomForestRegressorCustom(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        max_features=max_features,
                        random_state=42
                    )
                    rf_regressor.fit(X_train, y_train)
                    y_pred = rf_regressor.predict(X_test)
                    
                    mse = mean_squared_error(y_test, y_pred)

                    if mse < best_score:
                        best_score = mse
                        best_params = {
                            'n_estimators': n_estimators,
                            'max_depth': max_depth,
                            'min_samples_split': min_samples_split,
                            'max_features': max_features
                        }

    print(f"Best parameters: {best_params}")
    print(f"Best MSE: {best_score:.2f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'max_features': [None, 3, 5]
}

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_params_regression = grid_search_rf_regression(X_train, y_train, X_test, y_test, param_grid)


Best parameters: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2, 'max_features': 5}
Best MSE: 202.25


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'log2'}

In [45]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = RandomForestClassifierCustom(
    n_estimators=100, max_depth=10, min_samples_split=5, max_features="log2", random_state=42
)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

Accuracy: 0.9240, Recall: 0.9240, Precision: 0.9274, F1 Score: 0.9232


#### Задача регрессии

Лучшие параметры для регрессии: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2, 'max_features': 5}

In [47]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=2, max_features=5, random_state=42
)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 6.88, MSE: 202.25


### Выводы

После улучшения бейзлайна модели из sklearn продемонстрировали существенное повышение качества предсказаний как в задаче классификации, так и в задаче регрессии, превосходя показатели самописных реализаций. В классификации после оптимизации гиперпараметров sklearn-модель достигла значений Accuracy 0.9416 и F1 Score 0.9411, что указывает на ее высокую точность и сбалансированность. Самописная модель, хотя и показала улучшенные результаты по сравнению с первоначальными, с Accuracy 0.9240 и F1 Score 0.9232, все же уступила библиотечному аналогу, вероятно, из-за более ограниченных возможностей оптимизации. 

В задаче регрессии модели sklearn вновь оказались значительно эффективнее. После добавления нового признака и подбора гиперпараметров метрики MAE и MSE снизились до 2.64 и 33.98 соответственно, демонстрируя более точные и стабильные предсказания. Самописная модель, хотя и показала улучшение метрик (MAE: 6.88, MSE: 202.25), все же значительно отстала от sklearn-реализации. Это может быть связано с недостаточной реализацией некоторых ключевых аспектов случайного леса, таких как оптимизация количества признаков при разбиении или ограничений на глубину деревьев.

Таким образом, после улучшения бейзлайна модели sklearn продемонстрировали превосходство по всем ключевым метрикам, что делает их более подходящими для использования в практических задачах.